In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from ipynb.fs.defs.Local_search import new_route
from ipynb.fs.defs.Local_search import get_row
from ipynb.fs.defs.Local_search import get_column
from ipynb.fs.defs.Local_search import best_index
from ipynb.fs.defs.Local_search import schedule_time
from ipynb.fs.defs.Local_search import find_best_position
from ipynb.fs.defs.Local_search import cities_list
#table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
#time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
#fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
#fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";", encoding = "ISO-8859-1")
fur1 = pd.read_csv("../Dati/turni_f/2-opt1_shifts.csv", delimiter=";", encoding = "ISO-8859-1")
fur2 = pd.read_csv("../Dati/turni_f/2-opt2_shifts.csv", delimiter=";", encoding = "ISO-8859-1")

In [2]:
#Given a list of cities it return False if "to_insert" is already present
#Return True if "to_insert" isn't present and it will give the index of the best position
def check_insert(schedule_day_list, to_insert, dframe):
    if to_insert in schedule_day_list:
        return False, -1
    else:
        #print(schedule_day_list)
        schedule_day_list = cities_list(new_route(schedule_day_list, dframe))
        to_return = best_index(schedule_day_list, new_route(schedule_day_list, dframe), to_insert, dframe)
        return True, to_return

#Given: city list wich to_insert come, the city "to_insert" to be moved, 
#the city list where "to_insert" will be insered, the index of the best position and a dataframe
#the function return the improvement
#If negative the exchange mustn't be done, if positive you gain some time
def improvement(f_l, to_insert, t_l, index, dframe):
    from_list = f_l[:]
    from_time = sum(new_route(from_list, dframe).values)
    from_list.remove(to_insert)
    from_time_removed = sum(new_route(from_list, dframe).values)
    
    to_list = t_l[:]
    to_time = sum(new_route(to_list, dframe).values)
    to_list.insert(index, to_insert)
    to_time_insert = sum(new_route(to_list, dframe).values)
    
    return (from_time-from_time_removed) + (to_time-to_time_insert)

In [3]:
l = list(fur1["lunedi"].dropna())
m = list(fur1["martedi"].dropna())
i = l[3]
b, pos = check_insert(m, i, dft)
if b:
    print(improvement(l, i, m, pos,dft))

11.0


In [4]:
def get_best_improvement(days, veichle, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_f_list):
    for inner_day in days:
        if(veichle==1):
            curr_to_list = list(fur1[inner_day].dropna())
        else:
            curr_to_list = list(fur2[inner_day].dropna())
           
        if inner_day == day:
            continue
        else:
            boo, index = check_insert(curr_to_list, to_insert, dft)

        if boo:
            local_improvement = improvement(curr_f_list, to_insert, curr_to_list, index, dft)
        else:
            continue

        #If there's an improvement we update "best" variables, in other case we skip to next day
        if local_improvement<0:
            continue
        elif local_improvement>best_improvement:
            best_improvement = local_improvement
            best_day = inner_day
            best_veichle = veichle
            best_idx = index
        else:
            continue
    return best_improvement, best_day, best_veichle, best_idx

In [21]:
#For every day of every veichle we get every Location
#We compare the improvement removing this location from the day/veichle routing list and inserting it 
#in the other day/veichle routing lists.
#The location will be moved only if the improvement is positive 
#In case of different positive improvement we move in the higher
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
short_week = ["martedi", "mercoledi", "giovedi"]
fur1_copy = pd.DataFrame({})
fur1_copy['col'] = pd.Series([0]*50)
fur1_copy = fur1_copy.join(fur1)

fur2_copy = pd.DataFrame({})
fur2_copy['col'] = pd.Series([0]*50)
fur2_copy = fur2_copy.join(fur2)
for day in week:
    curr_from_list = list(fur1_copy[day].dropna())
    to_insert_list = list(fur1_copy[day].dropna())
    cycles = len(to_insert_list)
    #For every location in this schedule
    for i in range(1, cycles-1):
        print(to_insert_list)
        to_insert = to_insert_list.pop(i)
        
        #to_insert = to_insert_list.pop(0)
        
        best_improvement = -1000
        best_day = ""
        best_veichle = ""
        best_idx = ""
        #Here we must compare every possible insert in other days
        #Cycle for first veichle
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(week, 1, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list)
        best_improvement, best_day, best_veichle, best_idx = get_best_improvement(short_week, 2, best_improvement, best_day, best_veichle, best_idx, to_insert, curr_from_list)
        #If best_day isn't modified where isn't an improvement for every day/veihe
        if(best_day==""):
            continue
        else:
            #Remove from origin list and insert into target list
            curr_from_list.remove(to_insert)
            if(best_veichle==1):
                temp_list = list(fur1_copy[best_day])
                temp_list.insert(best_idx, to_insert)
                if(len(temp_list)>len(fur1_copy['col'])):
                    temp_list.pop(len(temp_list)-1)
                #fur1_copy[best_day] = pd.Series(temp_list)
                fur1_copy[best_day] =  fur1_copy[best_day].replace(list(fur1_copy[best_day]), temp_list)
            elif(best_veichle==2):
                temp_list = list(fur2_copy[best_day])
                temp_list.insert(best_idx, to_insert)
                if(len(temp_list)>len(fur2_copy['col'])):
                    temp_list.pop(len(temp_list)-1)
                #fur2_copy[best_day] = pd.Series(temp_list)
                fur2_copy[best_day] =  fur2_copy[best_day].replace(list(fur2_copy[best_day]), temp_list)
fur1_copy   

['Cremona, Sesto 39', 'Sesto ed Uniti, Roma, 4', 'Grumello Cremonese ed Uniti, Roma, 2', 'Pizzighettone, Giuseppe Garibaldi, 18', 'Castelleone, Roma, 67', 'Montodine,  XXV Aprile, 10', 'Ripalta Cremasca, Roma, 5', 'Ricengo, Guglielmo Marconi', 'Capralba, Piave, 2', 'Sergnano, IV Novembre,  10', 'Casale Cremasco', 'Camisano, Trieste, 57', 'Soncino, Dante Alighieri, 23', 'Trigolo, Roma, 26', 'Soresina, Giacomo Matteotti, 6', 'Ostiano, Mazzini, 1']
['Cremona, Sesto 39', 'Grumello Cremonese ed Uniti, Roma, 2', 'Pizzighettone, Giuseppe Garibaldi, 18', 'Castelleone, Roma, 67', 'Montodine,  XXV Aprile, 10', 'Ripalta Cremasca, Roma, 5', 'Ricengo, Guglielmo Marconi', 'Capralba, Piave, 2', 'Sergnano, IV Novembre,  10', 'Casale Cremasco', 'Camisano, Trieste, 57', 'Soncino, Dante Alighieri, 23', 'Trigolo, Roma, 26', 'Soresina, Giacomo Matteotti, 6', 'Ostiano, Mazzini, 1']
['Cremona, Sesto 39', 'Grumello Cremonese ed Uniti, Roma, 2', 'Castelleone, Roma, 67', 'Montodine,  XXV Aprile, 10', 'Ripalta C

IndexError: pop index out of range